In [1]:
from pulp import *

### We have 2 warehouses, 5-6 retail/customers. Each customer has their own demand and cost associated with transportation from each of the 2 warehouses. The optimization here is to make sure the demand for every customer is met from a warehouse that is cost-effective.

In [2]:
Warehouses = ["A", "B"]
supply = {
    "A": 1000, # 1000 products in warehouse A
    "B": 4000
}

Customers = ["1", "2", "3", "4", "5"]

demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}

costs = [
    [2, 4, 5, 2, 1],
    [3, 1, 3, 2, 3]
]

In [3]:
cost = makeDict([Warehouses, Customers], costs, 0)

# cost = {
#     'A': {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1},
#     'B': {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3}
# }
cost

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'A': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}),
             'B': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3})})

In [4]:
prob = LpProblem("WarehousePlanning", LpMinimize)

Routes = [(w, c) for w in Warehouses for c in Customers]
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5')]

In [7]:
vars = LpVariable.dicts("Route", (Warehouses, Customers), lowBound=0, upBound=None, cat=LpInteger)
vars

{'A': {'1': Route_A_1,
  '2': Route_A_2,
  '3': Route_A_3,
  '4': Route_A_4,
  '5': Route_A_5},
 'B': {'1': Route_B_1,
  '2': Route_B_2,
  '3': Route_B_3,
  '4': Route_B_4,
  '5': Route_B_5}}

In [8]:
prob += lpSum([vars[w][c] * cost[w][c] for (w, c) in Routes]), "Sum_of_Transporting_Costs"

In [9]:
for w in Warehouses:
    prob += lpSum([vars[w][c] for c in Customers]) <= supply[w], "Sum_of_Products_out_of_Warehouse_%s" % w
    
for c in Customers:
    prob += lpSum([vars[w][c] for w in Warehouses]) >= demand[c], "Sum_of_Products_into_Customer_%s" % c

In [11]:
prob.writeLP("WarehousingProblem.lp")

[Route_A_1,
 Route_A_2,
 Route_A_3,
 Route_A_4,
 Route_A_5,
 Route_B_1,
 Route_B_2,
 Route_B_3,
 Route_B_4,
 Route_B_5]

In [13]:
prob.solve(pulp.PULP_CBC_CMD(msg=False))
for v in prob.variables():
    print(v.name, "=", v.varValue)

Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0


In [14]:
print("Total Cost of Transportation = ", value(prob.objective))

Total Cost of Transportation =  8600.0
